# Colab Notebook: Agentic Patterns with LangGraph

# Install and Import

In [ ]:
# Install required libraries
!pip install -q -U langgraph langchain-openai langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.9/469.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.3/208.3 kB 17.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires langchain-core<1.0.0,>=0.3.72, but you have langchain-core 1.0.3 which is incompatible.


# Reactive Agent (Single-Step Response)

In [ ]:
"""
LANGRAPH AGENT: MATH CALCULATOR
================================

A simple math tutor agent that solves arithmetic problems in 2 steps:

STEP 1 - LLM NODE:
   - Receives user's math question
   - Extracts the numeric expression (e.g., "8 * (5 + 2)")
   - Uses agent role to guide extraction

STEP 2 - CALCULATOR NODE:
   - Takes the expression from LLM
   - Evaluates it using Python's calculator tool
   - Returns the final result

FLOW: User Question → LLM (extract expression) → Calculator (compute) → Result

Components:
- AgentState: Stores input question and output result
- AGENT_ROLE: Defines the math tutor behavior
- calculator tool: Performs arithmetic operations
- StateGraph: Connects LLM and Calculator in sequence
"""

'\nLANGRAPH AGENT: MATH CALCULATOR\n================================\n\nA simple math tutor agent that solves arithmetic problems in 2 steps:\n\nSTEP 1 - LLM NODE:\n   - Receives user\'s math question\n   - Extracts the numeric expression (e.g., "8 * (5 + 2)")\n   - Uses agent role to guide extraction\n\nSTEP 2 - CALCULATOR NODE:\n   - Takes the expression from LLM\n   - Evaluates it using Python\'s calculator tool\n   - Returns the final result\n\nFLOW: User Question → LLM (extract expression) → Calculator (compute) → Result\n\nComponents:\n- AgentState: Stores input question and output result\n- AGENT_ROLE: Defines the math tutor behavior\n- calculator tool: Performs arithmetic operations\n- StateGraph: Connects LLM and Calculator in sequence\n'

In [ ]:
#  Imports
from langgraph.graph import StateGraph, END, START
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
import os

os.environ["OPENAI_API_KEY"] = ""



In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

# --- Define schema ---
class AgentState(dict):
    input: str
    output: str

AGENT_ROLE = """You are a math tutor assistant.
Your job is to extract mathematical expressions and help solve them."""

# --- Define a tool ---
@tool
def calculator(expression: str) -> str:
    """Performs basic arithmetic calculations."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

# --- Initialize LLM ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- LLM node ---
def llm_node(state: AgentState):
    """LLM decides the expression to calculate."""
    question = state["input"]
    # Use the agent role to guide the LLM's behavior
    reasoning = llm.invoke([
        {"role": "system", "content": AGENT_ROLE},
        {"role": "user", "content": f"Extract ONLY the mathematical expression from: {question}\nReturn ONLY the expression with numbers and operators, nothing else. No words, no explanations."}
    ])
    # Clean the output - remove any extra text or quotes
    expression = reasoning.content.strip().strip('"').strip("'")
    print(f"Extracted expression: {expression}")
    return {"output": expression}

# --- Tool node ---
def tool_node(state: AgentState):
    """Tool executes the computation requested by the LLM."""
    result = calculator.invoke({"expression": state["output"]})
    return {"output": f"The result is {result}"}

# ---  Build the Agent Graph ---
# LangGraph lets us connect different components (nodes) together
# into a directed flow — like a neural network for decision and action.

graph = StateGraph(AgentState)  # Initialize the graph with the state structure (input/output fields)

# Add each node in the workflow
graph.add_node("LLM", llm_node)           #  Node 1: The LLM — interprets the question, extracts the math
graph.add_node("Calculator", tool_node)   #  Node 2: The Tool — performs the actual computation

# Define the flow (edges) between nodes
graph.add_edge(START, "LLM")              # Start → LLM (entry point)
graph.add_edge("LLM", "Calculator")       # LLM output is sent to the Calculator tool
graph.add_edge("Calculator", END)         # End after the calculator returns a result

# --- Compile and run ---
agent = graph.compile()
result = agent.invoke({"input": "What is 8 * (5 + 2)?"})
print(result["output"])

Extracted expression: 8 * (5 + 2)
The result is 56


# ReAct Agent — Web Search + Summarization

In [ ]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.2 MB/s eta 0:00:00


In [ ]:
"""
LANGRAPH AGENT: SEARCH ASSISTANT
=================================

This agent demonstrates a simple ReAct-style workflow with 3 phases:

STEP 1 - REASONING (🧠):
   - Takes user input
   - Uses LLM to extract clean search keywords
   - Transitions to search phase

STEP 2 - SEARCHING (🔍):
   - Takes the keywords from step 1
   - Generates a Google search URL
   - Transitions to reflection phase

STEP 3 - REFLECTING (🪞):
   - Uses LLM to summarize what the user will find
   - Returns final answer
   - Agent completes

FLOW: User Input → Reasoning → Search → Reflection → Final Answer

The agent uses:
- StateGraph: Manages workflow between nodes
- Conditional edges: Routes based on phase (reason → search → reflect → done)
- LLM calls: For reasoning and reflection
- Tool: For generating search URLs
"""

'\nLANGRAPH AGENT: SEARCH ASSISTANT\n=================================\n\nThis agent demonstrates a simple ReAct-style workflow with 3 phases:\n\nSTEP 1 - REASONING (🧠):\n   - Takes user input\n   - Uses LLM to extract clean search keywords\n   - Transitions to search phase\n\nSTEP 2 - SEARCHING (🔍):\n   - Takes the keywords from step 1\n   - Generates a Google search URL\n   - Transitions to reflection phase\n\nSTEP 3 - REFLECTING (🪞):\n   - Uses LLM to summarize what the user will find\n   - Returns final answer\n   - Agent completes\n\nFLOW: User Input → Reasoning → Search → Reflection → Final Answer\n\nThe agent uses:\n- StateGraph: Manages workflow between nodes\n- Conditional edges: Routes based on phase (reason → search → reflect → done)\n- LLM calls: For reasoning and reflection\n- Tool: For generating search URLs\n'

In [ ]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

@tool
def google_search(query: str) -> str:
    """Generate a Google search URL."""
    return f"https://www.google.com/search?q={query.replace(' ', '+')}"

class AgentState(dict):
    input: str
    query: str
    url: str
    output: str
    phase: str

llm = ChatOpenAI(model="gpt-4o", temperature=0)

AGENT_ROLE = """You are a research assistant that helps users find information.
Your job is to:
1. Understand the user's question
2. Generate a search query
3. Provide a helpful summary of what they'll find"""

def llm_node(state: AgentState):
    phase = state.get("phase", "reason")

    if phase == "reason":
        print("\n🧠 PHASE 1: REASONING")
        print(f"   Input: {state['input']}")
        response = llm.invoke([
            {"role": "system", "content": AGENT_ROLE},
            {"role": "user", "content": f"Extract only the search keywords for: {state['input']}\nReturn ONLY the keywords, nothing else."}
        ])
        query = response.content.strip().strip('"')
        print(f"   Query: {query}")
        return {**state, "query": query, "phase": "search"}

    if phase == "reflect":
        print("\n🪞 PHASE 3: REFLECTING")
        print(f"   Analyzing results from search")
        summary = llm.invoke([
            {"role": "system", "content": AGENT_ROLE},
            {"role": "user", "content": f"The user searched for: '{state['input']}'\nProvide a 2-3 sentence summary of what information they would find about this topic."}
        ])
        print(f"   Summary generated ✓")
        return {**state, "output": summary.content, "phase": "done"}

    return state

def search_node(state: AgentState):
    print("\n🔍 PHASE 2: SEARCHING")
    url = google_search.invoke({"query": state["query"]})
    print(f"   URL: {url}")
    return {**state, "url": url, "phase": "reflect"}
    # Keep all existing state values unchanged, but update "url" with new search URL
    # and change "phase" to "reflect" (next step after search completes)

# Build graph
graph = StateGraph(AgentState)  # Initialize graph with our state schema
graph.add_node("LLM", llm_node)  # Add node for reasoning and reflection
graph.add_node("Search", search_node)  # Add node for search action

# Define edges (connections between nodes)
graph.add_edge(START, "LLM")  # Entry point: always start at LLM node
graph.add_edge("Search", "LLM")  # After search, always return to LLM for reflection

# Conditional routing from LLM node based on phase
graph.add_conditional_edges(
    "LLM",  # From LLM node
    lambda s: "END" if s.get("phase") == "done" else "Search",  # Decision function: if done → END, else → Search
    {"END": END, "Search": "Search"}  # Map decisions to actual destinations

)
# Run
agent = graph.compile()
result = agent.invoke({"input": "latest trends in generative AI 2025"})

print("\n" + "=")
print("FINAL RESULT")
print("=")
print(f"ANSWER: {result['output']}")
print(f"URL: {result['url']}")


🧠 PHASE 1: REASONING
   Input: latest trends in generative AI 2025
   Query: latest trends generative AI 2025

🔍 PHASE 2: SEARCHING
   URL: https://www.google.com/search?q=latest+trends+generative+AI+2025

🪞 PHASE 3: REFLECTING
   Analyzing results from search
   Summary generated ✓

=
FINAL RESULT
=
ANSWER: By 2025, the latest trends in generative AI are expected to include advancements in multimodal AI systems that can seamlessly integrate text, image, and audio generation. There will likely be a focus on improving the ethical and responsible use of AI, with enhanced transparency and bias mitigation techniques. Additionally, generative AI is anticipated to play a significant role in personalized content creation, virtual reality, and the development of more sophisticated AI-driven tools for creative industries.
URL: https://www.google.com/search?q=latest+trends+generative+AI+2025
